# GEE Setup

In [1]:
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-natthaphonwin')

In [ ]:
import pandas as pd
from datetime import datetime

# Find small patches that are forest

Use geometry of HLS data for correctly match MGRS tiles

In [3]:
def get_geometry(tile_id):
  hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
  collection = hlss30.filter(ee.Filter.eq('MGRS_TILE_ID', tile_id))
  max_coverage = collection.aggregate_max('SPATIAL_COVERAGE').getInfo()
  union_geom = collection.geometry().dissolve()
  centroid = union_geom.centroid(maxError=1)
  center_long, center_lat = centroid.coordinates().getInfo()
  return center_long, center_lat, union_geom

In [4]:
def create_mgrs_patch(mgrs_code, center_long, center_lat, split=True):
  # Compute big tile
  min_lon = center_long - 1 / 2
  max_lon = center_long + 1 / 2
  min_lat = center_lat - 1 / 2
  max_lat = center_lat + 1 / 2
  features = []

  if split:
    # Compute and create small tiles
    n_rows = 7
    n_cols = 7
    small_width = (max_lon - min_lon) / n_cols
    small_height = (max_lat - min_lat) / n_rows

    for row in range(n_rows):
      for col in range(n_cols):
          small_min_lon = min_lon + col * small_width
          small_max_lon = small_min_lon + small_width
          small_min_lat = min_lat + row * small_height
          small_max_lat = small_min_lat + small_height

          rect = ee.Geometry.Rectangle([small_min_lon, small_min_lat, small_max_lon, small_max_lat])
          feature = ee.Feature(rect, {
              'tile_code': mgrs_code,
              'row': row,
              'col': col
          })
          features.append(feature)
  else:
    # Create only MGRS (large) tiles
    rect = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])
    feature = ee.Feature(rect, {
              'tile_code': mgrs_code,
    })
    features.append(feature)

  # Combine into a FeatureCollection
  fc = ee.FeatureCollection(features)

  return fc

In [5]:
def tile_visualize(tiles_list, split):
  fc_all = ee.FeatureCollection([])
  for tile in tiles_list:
    center_long, center_lat, geometry = get_geometry(tile)
    fc = create_mgrs_patch(mgrs_code=tile, center_long=center_long, center_lat=center_lat, split=split)
    fc_all = fc_all.merge(fc)
  Map = geemap.Map()
  Map.centerObject(fc_all, 8)
  Map.addLayer(fc_all, {'color': 'blue'}, 'MGRS Rectangles')
  return Map

In [6]:
tiles_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
              '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
              '47QMV', '47QNV']
map = tile_visualize(tiles_list=tiles_list, split=False)
map

Map(center=[18.719610623096614, 99.44074677249426], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
i = 11
print([tiles_list[i]])
map = tile_visualize(tiles_list=[tiles_list[i]], split=True)
map

['47QNV']


Map(center=[17.592037927106265, 99.51707213875416], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# Test HLS geometry and MGRS rectangles that I create
tile_id = tiles_list[0]
center_long, center_lat, geom = get_geometry(tile_id)
print(f"MGRS Tile: {tile_id}")
print(f"Center point (long, lat): {center_long}, {center_lat}")

fc = create_mgrs_patch(tile_id, center_long, center_lat)

Map = geemap.Map()
Map.centerObject(fc, 8)
Map.addLayer(geom, {'color': 'blue'}, 'Geometry from collection')
Map.addLayer(fc, {'color': 'red'}, 'My rectangle')
Map.addLayer(ee.Geometry.Point([center_long, center_lat]), {'color': 'green'}, 'Centroid')
Map

MGRS Tile: 47QLB
Center point (long, lat): 97.61808274918288, 19.395155248891207


Map(center=[19.394880276287438, 97.61808274918268], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
forest_df = pd.read_csv("forest_tile.csv")
forest_df["splited_tiles"] = 'T'+forest_df["MGRS Tiles"]+'_'+forest_df["Row"].astype(str)+'_'+forest_df["Column"].astype(str)
forest_df.head()

# Positive group

## Get dates

In [ ]:
def filter(i_date, f_date, cloud_coverage=20, spatial_coverage=50, bands=['B2', 'B3', 'B4', 'B8A', 'B11', 'B12'], tiles_list=None):
  hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
  if tiles_list is None:
    tiles_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
                  '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
                  '47QMV', '47QNV']
  hlss30_filter = hlss30.filter(ee.Filter.inList('MGRS_TILE_ID', tiles_list))
  hlss30_filter = hlss30_filter.filterDate(i_date, f_date)
  hlss30_filter = hlss30_filter.filter(ee.Filter.lte('CLOUD_COVERAGE', cloud_coverage))
  hlss30_filter = hlss30_filter.filter(ee.Filter.gte('SPATIAL_COVERAGE', spatial_coverage))
  hlss30_filter = hlss30_filter.select(bands)
  return hlss30_filter

In [ ]:
def get_pos_date(year, tile_list):
  # Febuary to April
  i_date = f"{year}-01"
  f_date = f"{year}-05"
  ee_tile_list = ee.List(tile_list)
  count = 0

  # Filter and sampling from i_date and f_date
  sampled_collection = filter(i_date=i_date, f_date=f_date)
  count += sampled_collection.size().getInfo()

  # Add date from timestamp
  def add_date(image):
      date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
      return image.set('date', date_str)
  sampled_collection = sampled_collection.map(add_date)

  # Get tiles on each date
  def tiles_each_date(date_str):
    images = sampled_collection.filter(ee.Filter.eq('date', date_str))
    tile_ids = ee.List(images.aggregate_array('MGRS_TILE_ID').distinct())

    tile_flags = ee.Dictionary.fromLists(
        ee_tile_list,
        ee_tile_list.map(lambda tile: tile_ids.contains(tile))
    )

    return tile_flags.set('date', date_str)

  unique_dates = sampled_collection.aggregate_array('date').distinct()
  results = unique_dates.map(tiles_each_date)
  output = results.getInfo()
  df = pd.DataFrame(output)

  return df

In [ ]:
tile_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
             '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
             '47QMV', '47QNV']

In [ ]:
# 2024
pos_2024 = get_pos_date(year=2024, tile_list=tile_list)
pos_2024.to_csv('pos_2024.csv', index=False)
pos_2024.shape

(56, 13)

In [ ]:
# 2023
pos_2023 = get_pos_date(year=2023, tile_list=tile_list)
pos_2023.to_csv('pos_2023.csv', index=False)
pos_2023.shape

(55, 13)

In [ ]:
# 2022
pos_2022 = get_pos_date(year=2022, tile_list=tile_list)
pos_2022.to_csv('pos_2022.csv', index=False)
pos_2022.shape

(39, 13)

In [ ]:
# 2021
pos_2021 = get_pos_date(year=2021, tile_list=tile_list)
pos_2021.to_csv('pos_2021.csv', index=False)
pos_2021.shape

(52, 13)

## Get tiles according hotspot with sampling

In [ ]:
hotspot_df = pd.read_csv("hotspot.csv")
tile_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
             '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
             '47QMV', '47QNV']
hotspot_df[tile_list] = hotspot_df[tile_list].astype(bool)
hotspot_df['tile_selected'] = hotspot_df[tile_list].apply(lambda row: [col for col in row.index if row[col]], axis=1)
hotspot_df.head()

,i_date,f_date,47QLA,47QLB,47QMA,47QMB,47QMV,47QNA,47QNB,47QNV,47QPA,47QPB,47QQA,47QQB,tile_selected
0,2021-02-05,2021-02-06,False,False,False,False,False,False,False,False,False,False,True,True,"[47QQB, 47QQA]"
1,2021-02-06,2021-02-07,True,True,False,False,False,False,False,False,False,False,False,False,"[47QLB, 47QLA]"
2,2021-02-13,2021-02-14,False,False,True,True,True,True,True,True,True,True,False,False,"[47QMB, 47QNB, 47QPB, 47QMA, 47QNA, 47QPA, 47Q..."
3,2021-03-18,2021-03-19,True,True,False,False,False,False,False,False,False,False,False,False,"[47QLB, 47QLA]"
4,2021-03-20,2021-03-21,False,False,True,True,True,True,True,True,True,True,False,False,"[47QMB, 47QNB, 47QPB, 47QMA, 47QNA, 47QPA, 47Q..."


In [ ]:
def get_ids_list(i_date, f_date, tile_id):
  def filter(i_date, f_date, cloud_coverage=20, spatial_coverage=50, bands=['B2', 'B3', 'B4', 'B8A', 'B11', 'B12'], tiles_list=None):
    hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
    if tiles_list is None:
      tiles_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
                    '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
                    '47QMV', '47QNV']
    hlss30_filter = hlss30.filter(ee.Filter.inList('MGRS_TILE_ID', tiles_list))
    hlss30_filter = hlss30_filter.filterDate(i_date, f_date)
    hlss30_filter = hlss30_filter.filter(ee.Filter.lte('CLOUD_COVERAGE', cloud_coverage))
    hlss30_filter = hlss30_filter.filter(ee.Filter.gte('SPATIAL_COVERAGE', spatial_coverage))
    hlss30_filter = hlss30_filter.select(bands)
    return hlss30_filter

  collection = filter(i_date, f_date, tiles_list=tile_id)
  id_list = collection.aggregate_array("system:id").getInfo()
  id_list = [s.replace("NASA/HLS/HLSS30/v002/", "") for s in id_list]
  return id_list

In [ ]:
id_list = []
for i in range(len(hotspot_df)):
  i_date = hotspot_df.iloc[i]["i_date"]
  f_date = hotspot_df.iloc[i]["f_date"]
  tile_selected = hotspot_df.iloc[i]["tile_selected"]
  id_list += get_ids_list(i_date, f_date, tile_selected)
len(id_list)

96

In [ ]:
burn_df = pd.DataFrame(id_list, columns=['ID'])
burn_df["MGRS_Tile"] = burn_df["ID"].str.split('_').str[0]
burn_df["Year"] = burn_df["ID"].str.slice(7, 11)
burn_df["Month"] = burn_df["ID"].str.slice(11, 13)
burn_df.head()

,ID,MGRS_Tile,Year,Month
0,T47QQA_20210205T033931,T47QQA,2021,02
1,T47QQB_20210205T033931,T47QQB,2021,02
2,T47QLA_20210206T035939,T47QLA,2021,02
3,T47QLB_20210206T035939,T47QLB,2021,02
4,T47QMA_20210213T034849,T47QMA,2021,02


In [ ]:
burn_df[["ID", "Year"]].groupby("Year").count()

,ID
Year,
2021,24
2022,24
2023,24
2024,24


In [ ]:
burn_df[["ID", "Month"]].groupby("Month").count()

,ID
Month,
01,19
02,15
03,60
04,2


In [ ]:
burn_df.to_csv("burn_tiles.csv")

# Negative group

In [ ]:
def filter(i_date, f_date, cloud_coverage=20, spatial_coverage=50, bands=['B2', 'B3', 'B4', 'B8A', 'B11', 'B12'], tiles_list=None):
  hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
  if tiles_list is None:
    tiles_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
                  '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
                  '47QMV', '47QNV']
  hlss30_filter = hlss30.filter(ee.Filter.inList('MGRS_TILE_ID', tiles_list))
  hlss30_filter = hlss30_filter.filterDate(i_date, f_date)
  hlss30_filter = hlss30_filter.filter(ee.Filter.lte('CLOUD_COVERAGE', cloud_coverage))
  hlss30_filter = hlss30_filter.filter(ee.Filter.gte('SPATIAL_COVERAGE', spatial_coverage))
  hlss30_filter = hlss30_filter.select(bands)
  return hlss30_filter

In [ ]:
neg_2021 = filter(i_date='2021-07-01', f_date='2021-12-31', cloud_coverage=30)
neg_2022 = filter(i_date='2022-07-01', f_date='2022-12-31', cloud_coverage=30)
neg_2023 = filter(i_date='2023-07-01', f_date='2023-12-31', cloud_coverage=30)
neg_2024 = filter(i_date='2024-07-01', f_date='2024-12-31', cloud_coverage=30)
print(f"Total negative at 2021: {neg_2021.size().getInfo()}")
print(f"Total negative at 2022: {neg_2022.size().getInfo()}")
print(f"Total negative at 2023: {neg_2023.size().getInfo()}")
print(f"Total negative at 2024: {neg_2024.size().getInfo()}")

Total negative at 2021: 76
Total negative at 2022: 75
Total negative at 2023: 78
Total negative at 2024: 85


In [ ]:
def sampling_image(image_collection, n_sample):
  tile_list = ['47QLB', '47QMB', '47QNB', '47QPB', '47QQB',
               '47QLA', '47QMA', '47QNA', '47QPA', '47QQA',
               '47QMV', '47QNV']

  image_collection = image_collection.randomColumn('random', distribution='normal')
  sampled_images = []
  for tile in tile_list:
    filtered = image_collection.filter(ee.Filter.eq('MGRS_TILE_ID', tile)).sort('random')
    n = min(n_sample, filtered.size().getInfo())
    # print(f"Sample {tile} with {n} patches")
    for i in range(n):
        image = ee.Image(filtered.toList(n_sample).get(i))
        sampled_images.append(image)
  sampled_collection = ee.ImageCollection.fromImages(sampled_images)

  id_list = sampled_collection.aggregate_array("system:id").getInfo()
  id_list = [s.replace("NASA/HLS/HLSS30/v002/", "") for s in id_list]

  return id_list

In [ ]:
neg_2021_sampled = sampling_image(neg_2021, 2)
neg_2022_sampled = sampling_image(neg_2022, 2)
neg_2023_sampled = sampling_image(neg_2023, 2)
neg_2024_sampled = sampling_image(neg_2024, 2)

nonburn_tiles = neg_2021_sampled + neg_2022_sampled + neg_2023_sampled + neg_2024_sampled
len(nonburn_tiles)

96

In [ ]:
unburn_df = pd.DataFrame(nonburn_tiles, columns=['ID'])
unburn_df["MGRS_Tile"] = unburn_df["ID"].str.split('_').str[0]
unburn_df["Year"] = unburn_df["ID"].str.slice(7, 11)
unburn_df["Month"] = unburn_df["ID"].str.slice(11, 13)
unburn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         96 non-null     object
 1   MGRS_Tile  96 non-null     object
 2   Year       96 non-null     object
 3   Month      96 non-null     object
dtypes: object(4)
memory usage: 3.1+ KB


In [ ]:
unburn_df[["ID", "Year"]].groupby("Year").count()

,ID
Year,
2021,24
2022,24
2023,24
2024,24


In [ ]:
unburn_df[["ID", "Month"]].groupby("Month").count()

,ID
Month,
07,4
08,1
10,7
11,29
12,55


In [ ]:
unburn_df.to_csv("unburn_tiles.csv")

# Merge positive and negative group

In [ ]:
unburn_df = pd.read_csv("unburn_tiles.csv")
unburn_df = unburn_df.iloc[:, 1:]
unburn_df.rename(columns={"ID": "Unburned ID"}, inplace=True)
unburn_df.drop(columns=["Month"], inplace=True)
unburn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unburned ID  96 non-null     object
 1   MGRS_Tile    96 non-null     object
 2   Year         96 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [ ]:
unburn_df["group_id"] = (
    unburn_df
    .groupby(["Year", "MGRS_Tile"])
    .cumcount() + 1
)
unburn_df.sort_values(by=["Year", "MGRS_Tile"])

,Unburned ID,MGRS_Tile,Year,group_id
10,T47QLA_20211228T040201,T47QLA,2021,1
11,T47QLA_20211108T035951,T47QLA,2021,2
0,T47QLB_20211218T040201,T47QLB,2021,1
1,T47QLB_20211208T040141,T47QLB,2021,2
12,T47QMA_20211210T035139,T47QMA,2021,1
...,...,...,...,...
79,T47QPB_20241114T034919,T47QPB,2024,2
90,T47QQA_20241121T033949,T47QQA,2024,1
91,T47QQA_20241111T033859,T47QQA,2024,2
80,T47QQB_20241012T033609,T47QQB,2024,1


In [ ]:
burn_df = pd.read_csv("burn_tiles.csv")
burn_df = burn_df.iloc[:, 1:]
burn_df.rename(columns={"ID": "Burned ID"}, inplace=True)
burn_df.drop(columns=["Month"], inplace=True)
burn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Burned ID  96 non-null     object
 1   MGRS_Tile  96 non-null     object
 2   Year       96 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [ ]:
burn_df["group_id"] = (
    burn_df
    .groupby(["Year", "MGRS_Tile"])
    .cumcount() + 1
)
burn_df.sort_values(by=["Year", "MGRS_Tile"])

,Burned ID,MGRS_Tile,Year,group_id
2,T47QLA_20210206T035939,T47QLA,2021,1
12,T47QLA_20210318T035539,T47QLA,2021,2
3,T47QLB_20210206T035939,T47QLB,2021,1
13,T47QLB_20210318T035539,T47QLB,2021,2
4,T47QMA_20210213T034849,T47QMA,2021,1
...,...,...,...,...
91,T47QPB_20240314T034531,T47QPB,2024,2
82,T47QQA_20240306T033609,T47QQA,2024,1
92,T47QQA_20240316T033539,T47QQA,2024,2
83,T47QQB_20240306T033609,T47QQB,2024,1


In [ ]:
merge_df = pd.merge(burn_df, unburn_df, on=["MGRS_Tile", "Year", "group_id"], how="inner")
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Burned ID    96 non-null     object
 1   MGRS_Tile    96 non-null     object
 2   Year         96 non-null     int64 
 3   group_id     96 non-null     int64 
 4   Unburned ID  96 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.9+ KB


In [ ]:
merge_df = merge_df[["MGRS_Tile", "Year", "Burned ID", "Unburned ID"]]
merge_df.rename(columns={"Burned ID": "Burned_ID", "Unburned ID": "Unburned_ID"}, inplace=True)
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MGRS_Tile    96 non-null     object
 1   Year         96 non-null     int64 
 2   Burned_ID    96 non-null     object
 3   Unburned_ID  96 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.1+ KB


In [ ]:
merge_df.to_csv("index.csv", index=False)

# Visualization example

In [ ]:
merge_df = pd.read_csv("index.csv")
merge_df = merge_df.iloc[:, 1:]
merge_df.head(20)

,Year,Burned_ID,Unburned_ID
0,2021,T47QQA_20210205T033931,T47QQA_20211202T034121
1,2021,T47QQB_20210205T033931,T47QQB_20211202T034121
2,2021,T47QLA_20210206T035939,T47QLA_20211228T040201
3,2021,T47QLB_20210206T035939,T47QLB_20211218T040201
4,2021,T47QMA_20210213T034849,T47QMA_20211210T035139
5,2021,T47QMB_20210213T034849,T47QMB_20211225T035151
6,2021,T47QMV_20210213T034849,T47QMV_20211215T035151
7,2021,T47QNA_20210213T034849,T47QNA_20211120T035039
8,2021,T47QNB_20210213T034849,T47QNB_20210822T034539
9,2021,T47QNV_20210213T034849,T47QNV_20211120T035039


In [ ]:
all_ids = merge_df["Burned_ID"].tolist() + merge_df["Unburned_ID"].tolist()
all_ids = list(set(all_ids))
all_ids = ["NASA/HLS/HLSS30/v002/" + a for a in all_ids]

In [ ]:
hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
hlss30_filter = hlss30.filter(ee.Filter.inList("system:id", all_ids))
hlss30_filter.size().getInfo()

192

In [ ]:
def normalize_band(image):
    min_list = [0.01, 0.01, 0.01, 0.01, 0.005, 0.005]
    max_list = [0.18, 0.18, 0.18, 0.5, 0.35, 0.3]

    band_names = ["B2", "B3", "B4", "B8A", "B11", "B12"]
    normalized_bands = []
    for band, min_val, max_val in zip(band_names, min_list, max_list):
        band_img = image.select(band)
        norm_band = band_img.subtract(min_val).divide(max_val - min_val).clamp(0,1).rename(band)
        normalized_bands.append(norm_band)

    normalized_image = ee.Image(normalized_bands)
    normalized_image = normalized_image.unmask(0)

    return normalized_image

In [ ]:
def create_layer(image, title, bands):
  vis_params = {
      'min': 0,
      'max':1,
      'bands': bands
  }
  layer = geemap.ee_tile_layer(image, vis_params, name=title)

  return layer

In [ ]:
merge_df[merge_df["Year"]==2022].sort_values(by=["Burned_ID"])

,Year,Burned_ID,Unburned_ID
44,2022,T47QLA_20220308T035621,T47QLA_20221024T035821
46,2022,T47QLA_20220313T035539,T47QLA_20220726T035551
45,2022,T47QLB_20220308T035621,T47QLB_20220726T035551
47,2022,T47QLB_20220313T035539,T47QLB_20221228T040159
31,2022,T47QMA_20220203T034951,T47QMA_20221110T035001
36,2022,T47QMA_20220305T034641,T47QMA_20221130T035121
32,2022,T47QMB_20220203T034951,T47QMB_20221228T040159
37,2022,T47QMB_20220305T034641,T47QMB_20221113T040021
33,2022,T47QMV_20220203T034951,T47QMV_20221220T035151
38,2022,T47QMV_20220305T034641,T47QMV_20221110T035001


In [ ]:
def plot_compare(merge_df, index, bands=["B4", "B3", "B2"]):
  burn_id = "NASA/HLS/HLSS30/v002/" + merge_df.iloc[index]["Burned_ID"]
  unburn_id = "NASA/HLS/HLSS30/v002/" + merge_df.iloc[index]["Unburned_ID"]
  mgrs_tile = merge_df.iloc[index]["Burned_ID"].split("_")[0][1:]
  print(f"MGRS tile: {mgrs_tile}")
  print(f"Burn id: {burn_id}")
  print(f"Unburn id: {unburn_id}")

  # Get feature collection for MGRS tile grid
  center_long, center_lat, geometry = get_geometry(mgrs_tile)
  fc = create_mgrs_patch(mgrs_code=mgrs_tile, center_long=center_long, center_lat=center_lat, split=True)

  hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
  hlss30 = hlss30.select(["B2", "B3", "B4", "B8A", "B11", "B12"])
  hlss30_burn = hlss30.filter(ee.Filter.eq("system:id", burn_id))
  hlss30_unburn = hlss30.filter(ee.Filter.eq("system:id", unburn_id))
  bounds  = hlss30_burn.geometry().bounds()
  center = bounds.centroid(maxError=100).coordinates().getInfo()

  normalized_burn = normalize_band(hlss30_burn.first())
  normalized_unburn = normalize_band(hlss30_unburn.first())

  burn_layer = create_layer(image=normalized_burn, title="Positive", bands=bands)
  nonburn_layer = create_layer(image=normalized_unburn, title="Negative", bands=bands)

  Map = geemap.Map(center=(center[1], center[0]), zoom=9)
  Map.split_map(nonburn_layer, burn_layer)
  Map.addLayer(fc, {'color': 'red'}, 'MGRS tiles')

  return Map

In [ ]:
rgb_bands = ['B4', 'B3', 'B2']
if_bands = ['B8A', 'B4', 'B3']
swir_bands = ['B12', 'B8A', 'B4']
agri_bands = ['B11', 'B8A', 'B2']

In [ ]:
map = plot_compare(merge_df, 44, rgb_bands)
map

MGRS tile: 47QLA
Burn id: NASA/HLS/HLSS30/v002/T47QLA_20220308T035621
Unburn id: NASA/HLS/HLSS30/v002/T47QLA_20221024T035821


Map(center=[18.49074012113793, 97.62396603377175], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
map = plot_compare(merge_df, 45, rgb_bands)
map

MGRS tile: 47QLB
Burn id: NASA/HLS/HLSS30/v002/T47QLB_20220308T035621
Unburn id: NASA/HLS/HLSS30/v002/T47QLB_20220726T035551


Map(center=[19.394388353437005, 97.61646081631893], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 31, rgb_bands)
map

MGRS tile: 47QMA
Burn id: NASA/HLS/HLSS30/v002/T47QMA_20220203T034951
Unburn id: NASA/HLS/HLSS30/v002/T47QMA_20221110T035001


Map(center=[18.49519298453586, 98.57713619749028], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
map = plot_compare(merge_df, 32, rgb_bands)
map

MGRS tile: 47QMB
Burn id: NASA/HLS/HLSS30/v002/T47QMB_20220203T034951
Unburn id: NASA/HLS/HLSS30/v002/T47QMB_20221228T040159


Map(center=[19.399422005895467, 98.68089770291832], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 33, rgb_bands)
map

MGRS tile: 47QMV
Burn id: NASA/HLS/HLSS30/v002/T47QMV_20220203T034951
Unburn id: NASA/HLS/HLSS30/v002/T47QMV_20221220T035151


Map(center=[17.591754516585006, 98.57357081157221], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 24, rgb_bands)
map

MGRS tile: 47QNA
Burn id: NASA/HLS/HLSS30/v002/T47QNA_20220129T035009
Unburn id: NASA/HLS/HLSS30/v002/T47QNA_20221220T035151


Map(center=[18.494950432224616, 99.52122795606891], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 25, rgb_bands)
map

MGRS tile: 47QNB
Burn id: NASA/HLS/HLSS30/v002/T47QNB_20220129T035009
Unburn id: NASA/HLS/HLSS30/v002/T47QNB_20221220T035151


Map(center=[19.398822932461723, 99.52412100063411], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 26, rgb_bands)
map

MGRS tile: 47QNV
Burn id: NASA/HLS/HLSS30/v002/T47QNV_20220129T035009
Unburn id: NASA/HLS/HLSS30/v002/T47QNV_20221220T035151


Map(center=[17.591533258263688, 99.51849575135536], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 27, rgb_bands)
map

MGRS tile: 47QPA
Burn id: NASA/HLS/HLSS30/v002/T47QPA_20220129T035009
Unburn id: NASA/HLS/HLSS30/v002/T47QPA_20221105T034929


Map(center=[18.49044275791509, 100.46855392708646], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 28, rgb_bands)
map

MGRS tile: 47QPB
Burn id: NASA/HLS/HLSS30/v002/T47QPB_20220129T035009
Unburn id: NASA/HLS/HLSS30/v002/T47QPB_20221215T035149


Map(center=[19.39366487339884, 100.47655798806979], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
map = plot_compare(merge_df, 29, rgb_bands)
map

MGRS tile: 47QQA
Burn id: NASA/HLS/HLSS30/v002/T47QQA_20220131T034001
Unburn id: NASA/HLS/HLSS30/v002/T47QQA_20221227T034151


Map(center=[18.480428833040005, 101.41486515460774], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:
map = plot_compare(merge_df, 30, rgb_bands)
map

MGRS tile: 47QQB
Burn id: NASA/HLS/HLSS30/v002/T47QQB_20220131T034001
Unburn id: NASA/HLS/HLSS30/v002/T47QQB_20221122T034049


Map(center=[19.382967577550506, 101.42796581602462], controls=(ZoomControl(options=['position', 'zoom_in_text'…

# Export Image

In [ ]:
hlss30 = ee.ImageCollection('NASA/HLS/HLSS30/v002')
hlss30 = hlss30.select(["B2", "B3", "B4", "B8A", "B11", "B12"])
burn_id = ["NASA/HLS/HLSS30/v002/" + a for a in merge_df["Burned_ID"]]
unburn_id = ["NASA/HLS/HLSS30/v002/" + a for a in merge_df["Unburned_ID"]]

hlss30_burn = hlss30.filter(ee.Filter.inList("system:id", burn_id))
hlss30_unburn = hlss30.filter(ee.Filter.inList("system:id", unburn_id))
print(f"Burned size: {hlss30_burn.size().getInfo()}")
print(f"Unburned size: {hlss30_unburn.size().getInfo()}")

Burned size: 96
Unburned size: 96


In [ ]:
def export(image_collection, name, folder):
  def export_image(image, name, folder):
      no_data_val = 0.0
      image = image.unmask(no_data_val)
      export_task = ee.batch.Export.image.toDrive(
          image = image.toFloat(),
          description = f"{name}_{image.get('id').getInfo()}",
          folder = folder,
          fileNamePrefix = image.get('id').getInfo(),
          fileFormat = 'GeoTIFF',
          region = image.geometry(),
          scale=30,
          formatOptions={'noData': no_data_val},
      )
      export_task.start()

  def normalize_band(image):
    min_list = [0.01, 0.01, 0.01, 0.01, 0.005, 0.005]
    max_list = [0.18, 0.18, 0.18, 0.5, 0.35, 0.3]

    # Normalize
    band_names = ["B2", "B3", "B4", "B8A", "B11", "B12"]
    normalized_bands = []
    for band, min_val, max_val in zip(band_names, min_list, max_list):
        band_img = image.select(band)
        norm_band = band_img.subtract(min_val).divide(max_val - min_val)
        normalized_bands.append(norm_band)

    # Combine to img
    normalized_image = ee.Image.cat(normalized_bands).rename(band_names)
    normalized_image = normalized_image.clamp(0,1)
    normalized_image = normalized_image.unmask(0.0)
    normalized_image = normalized_image.set("id", image.id())

    return normalized_image

  # Exports
  bands = ["B2", "B3", "B4", "B8A", "B11", "B12"]
  for index, image in enumerate(image_collection.toList(image_collection.size()).getInfo()):
  # for index, image in enumerate(image_collection.toList(1).getInfo()):
    image_ee = ee.Image(image['id']).select(bands)
    image_ee_normalized = normalize_band(image_ee)
    export_image(image_ee_normalized, name, folder)

In [ ]:
export(image_collection=hlss30_burn, name="Pos", folder="HLS_Thai_Positive")

In [ ]:
export(image_collection=hlss30_unburn, name="Neg", folder="HLS_Thai_Negative")